In [1]:
import pandas as pd
import pdb
import numpy as np
import itertools
from utils import *
from snakemake.io import expand
import yaml
import cerberus

In [2]:
config_file = 'workflow/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [27]:
species = 'human'
ca_h5 = 'workflow/'+expand(config['lr']['ca'], species=species)[0]
meta_file = 'workflow/'+expand(config['lr']['meta'], species=species)[0]
encode_meta_file = 'workflow/'+expand(config['lr']['encode_meta'], species=species)[0]
filt_ab = 'workflow/'+expand(config['lr']['filt_ab'], species=species)[0]

In [28]:
min_tpm = 1

In [49]:
# get dataset<->experiment<->encode biosample shorthand

meta = pd.read_csv(meta_file, sep='\t')
meta = meta[['ENCODE_experiment_id', 'dataset']].rename({'ENCODE_experiment_id': 'Experiment accession'}, axis=1)

enc_meta = process_encode_metadata(encode_meta_file)
enc_meta = enc_meta[['Experiment accession', 'biosamp', 'biorep']].drop_duplicates()

meta = meta.merge(enc_meta, how='left', on='Experiment accession')
meta = meta[['dataset', 'biosamp', 'biorep']]
meta.set_index('dataset', inplace=True)

In [50]:
df = pd.read_csv(filt_ab, sep='\t')
df = add_feat(df,
              'annot_transcript_id', 
              kind='tss', 
              as_index=True)

In [51]:
drop_cols = ['gene_ID', 'transcript_ID',
             'annot_gene_id', 'annot_transcript_id',
             'annot_gene_name', 'annot_transcript_name',
             'n_exons', 'length', 'gene_novelty',
             'transcript_novelty', 'ISM_subtype']
df.drop(drop_cols, axis=1, inplace=True)          

In [52]:
df = df.reset_index()
df = df.groupby('tss').sum()

In [53]:
df.head()

,h9_neural_crest_1_1,a673_1_1,h9_chondro_1_3,hl60_m2_24hr_1_2,h9_neural_crest_1_2,cardiac_septum_1_1,h9_panc_progen_1_2,lower_lobe_of_left_lung_2_1,h9_panc_beta_1_1,huvec_1_2,...,right_cardiac_atrium_2_1,right_ventricle_myocardium_superior_1_1,pc9_1_1,h9_1_2,pgp1_endo_1_1,k562_2_1,ovary_1_1,hl60_m1_72hr_1_1,k562_3_2,posterior_vena_cava_2_1
tss,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003_1,265,119,40,0,322,4,1269,28,603,162,...,10,15,197,569,79,1,330,0,0,6
ENSG00000000003_2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000000005_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000000005_3,0,0,0,0,0,1,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000000419_1,57,57,13,72,102,23,56,4,98,73,...,12,19,78,134,30,72,19,25,43,6


In [54]:
df = df.transpose()
df['total'] = df.sum(axis=1)

In [55]:
df = df.divide(df.total, axis=0)*1e6

In [56]:
df.drop('total', axis=1, inplace=True)

In [57]:
df = df>=min_tpm

In [58]:
df.head()

tss,ENSG00000000003_1,ENSG00000000003_2,ENSG00000000005_2,ENSG00000000005_3,ENSG00000000419_1,ENSG00000000419_8,ENSG00000000457_1,ENSG00000000457_2,ENSG00000000457_3,ENSG00000000460_1,...,ENSG00000285972_1,ENSG00000285976_1,ENSG00000285976_2,ENSG00000285978_1,ENSG00000285980_2,ENSG00000285988_1,ENSG00000285988_2,ENSG00000285988_3,ENSG00000285988_4,ENSG00000285991_1
h9_neural_crest_1_1,True,False,False,False,True,False,True,False,False,True,...,False,True,True,False,False,False,False,False,False,False
a673_1_1,True,False,False,False,True,False,True,False,False,True,...,False,True,True,False,False,False,False,False,False,True
h9_chondro_1_3,True,False,False,False,True,False,True,False,False,True,...,False,True,True,False,False,False,False,False,False,False
hl60_m2_24hr_1_2,False,False,False,False,True,False,False,False,False,False,...,False,True,True,False,False,False,False,False,False,False
h9_neural_crest_1_2,True,False,False,False,True,False,True,False,True,True,...,False,True,True,False,False,False,False,False,False,False


In [59]:
# add metadata so we can merge on biosample as dictated by encode
df = df.merge(meta, how='left', left_index=True, right_index=True)

In [60]:
df.reset_index(drop=True, inplace=True)
df = df.groupby(['biosamp', 'biorep']).max()

In [61]:
df.head()

ENSG00000000003_1  ENSG00000000003_2  ENSG00000000005_2  \
biosamp       biorep                                                            
a673          1                    True              False              False   
adrenal_gland 1                    True              False              False   
              2                    True              False              False   
              3                    True              False              False   
aorta         1                    True              False              False   

                      ENSG00000000005_3  ENSG00000000419_1  ENSG00000000419_8  \
biosamp       biorep                                                            
a673          1                   False               True              False   
adrenal_gland 1                   False               True              False   
              2                   False               True              False   
              3                   False               True              False   
aorta         1                   False               True              False   

                      ENSG00000000457_1  ENSG00000000457_2  ENSG00000000457_3  \
biosamp       biorep                                                            
a673          1                    True              False              False   
adrenal_gland 1                    True              False              False   
              2                    True              False              False   
              3                    True              False              False   
aorta         1                    True              False              False   

                      ENSG00000000460_1  ...  ENSG00000285972_1  \
biosamp       biorep                     ...                      
a673          1                    True  ...              False   
adrenal_gland 1                   False  ...              False   
              2                   False  ...              False   
              3                   False  ...              False   
aorta         1                   False  ...              False   

                      ENSG00000285976_1  ENSG00000285976_2  ENSG00000285978_1  \
biosamp       biorep                                                            
a673          1                    True               True              False   
adrenal_gland 1                    True               True              False   
              2                   False               True              False   
              3                   False               True              False   
aorta         1                   False               True              False   

                      ENSG00000285980_2  ENSG00000285988_1  ENSG00000285988_2  \
biosamp       biorep                                                            
a673          1                   False              False              False   
adrenal_gland 1                   False              False              False   
              2                   False              False              False   
              3                   False              False              False   
aorta         1                   False              False              False   

                      ENSG00000285988_3  ENSG00000285988_4  ENSG00000285991_1  
biosamp       biorep                                                           
a673          1                   False              False               True  
adrenal_gland 1                   False              False              False  
              2                   False              False              False  
              3                   False              False              False  
aorta         1                   False              False              False  

[5 rows x 76487 columns]

In [25]:
ca = cerberus.read(ca_h5)
tss = ca.tss.copy(deep=True)

In [26]:
tss.head()

,Chromosome,Start,End,Strand,Name,source,novelty,gene_id,tss
0,chr1,169794989,169795129,+,ENSG00000000460_1,"v40,v29,lapa,gtex,encode_cage,fantom_cage,enco...",Known,ENSG00000000460,1
1,chr1,169795358,169795459,+,ENSG00000000460_2,"v40,v29,lapa,pls",Known,ENSG00000000460,2
2,chr1,169794679,169794780,+,ENSG00000000460_3,"v40,v29,lapa,gtex,pls",Known,ENSG00000000460,3
3,chr1,169795870,169795971,+,ENSG00000000460_4,"v40,v29,pls",Known,ENSG00000000460,4
4,chr1,169661956,169662057,+,ENSG00000000460_5,"v40,v29,dels",Known,ENSG00000000460,5
